In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Importar y ejecutar el archivo utils.ipynb que contiene la definición de la clase Utils
%run '/home/roy/Scikitlearn/utils.ipynb'

# Intenta eliminar el atributo 'models' de la clase pd.DataFrame si existe
try:
    del pd.DataFrame.models
except AttributeError:
    pass

# Utilizamos el decorador de registro de extensiones de pandas para definir un nuevo accessor 'models'
pd.api.extensions.register_dataframe_accessor("models")
class Models:

    def __init__(self):
        self.reg = {
        'SVR' : SVR(),  # Inicializa un modelo SVR (Support Vector Regressor)
        'GRADIENT' : GradientBoostingRegressor()  # Inicializa un modelo GradientBoostingRegressor
    }

        self.params = {
        'SVR' : {
            'kernel' : ['linear', 'poly', 'rbf'],  # Diferentes kernels para SVR
            'gamma' : ['auto', 'scale'],  # Diferentes opciones para el parámetro gamma
            'C' : [1, 5, 10]  # Diferentes valores para el parámetro C
        },
        'GRADIENT' : {
            'loss' : ['squared_error', 'absolute_error'],  # Diferentes funciones de pérdida para GradientBoostingRegressor
            'learning_rate' : [0.01, 0.05, 0.1]  # Diferentes tasas de aprendizaje
        }
    }


    def grid_training(self, X,y):
    # Inicializar la mejor puntuación con un valor alto para comparar

        best_score = 999
        best_model = None
    # Iterar a través de los modelos y sus parámetros predefinidos

        for name, reg in self.reg.items():
        # Crear un objeto GridSearchCV para buscar los mejores hiperparámetros

            grid_reg = GridSearchCV(reg, self.params[name], cv=3).fit(X, y.values.ravel())
            # Obtener la mejor puntuación de la búsqueda de cuadrícula (valor absoluto)
            score = np.abs(grid_reg.best_score_)
            
            # Si la puntuación actual es mejor que la mejor puntuación anterior, actualizar

            if score < best_score:
                best_score = score
                best_model = grid_reg.best_estimator_
        
# Crear una instancia de la clase Utils para exportar el mejor modelo y su puntuación

        utils = Utils()
        utils.model_export(best_model, best_score)
